In [ ]:
# Load required standard modules
import numpy as np
from matplotlib import pyplot as plt

# Load required tudatpy modules
from tudatpy import constants
from tudatpy.interface import spice
from tudatpy import numerical_simulation
from tudatpy.numerical_simulation import environment
from tudatpy.numerical_simulation import environment_setup
from tudatpy.numerical_simulation import propagation_setup
from tudatpy.numerical_simulation import estimation, estimation_setup
from tudatpy.numerical_simulation.estimation_setup import observation
from tudatpy.astro.time_conversion import DateTime
from tudatpy.astro import element_conversion
from tudatpy.astro import frame_conversion
from tudatpy.util import result2array

In [ ]:
# Load spice kernels
spice.load_standard_kernels()

# Create default body settings for Sun, Earth, Moon, Mars, Venus
bodies_to_create = ["Sun", "Earth", "Moon", "Mars", "Venus"]
global_frame_origin = "Earth"
global_frame_orientation = "J2000"

# Create body settings
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)

# Replace Earth's default atmosphere with NRLMSISE-00
body_settings.get("Earth").atmosphere_settings = environment_setup.atmosphere.nrlmsise00()

# ✅ Earth gravity: EGM96 up to degree/order 50 (single arg caps both)
body_settings.get("Earth").gravity_field_settings = environment_setup.gravity_field.predefined_spherical_harmonic(
    environment_setup.gravity_field.egm96,   # <- note: 'egm96' constant, not an enum member path
    50                                       # maximum degree (and order)
)


In [ ]:
# Add empty settings for ISS before creating the system of bodies
body_settings.add_empty_settings("ISS")



In [ ]:
body_settings.get("ISS").constant_mass = 419725
reference_area_drag = 2500  # Average projection area of ISS
drag_coefficient = 2.2
aero_coefficient_settings = environment_setup.aerodynamic_coefficients.constant(
    reference_area_drag, [drag_coefficient, 0.0, 0.0]
)

# Add the aerodynamic interface to the body settings
body_settings.get("ISS").aerodynamic_coefficient_settings = aero_coefficient_settings



In [ ]:
# Create radiation pressure settings
reference_area_radiation = 2500  # Average projection area of ISS
radiation_pressure_coefficient = 2
occulting_bodies = dict()
occulting_bodies["Sun"] = ["Earth"]
radiation_pressure_settings = environment_setup.radiation_pressure.cannonball_radiation_target(
    reference_area_radiation, radiation_pressure_coefficient, occulting_bodies)

# Add the radiation pressure interface to the environment
body_settings.get("ISS").radiation_pressure_target_settings = radiation_pressure_settings

# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [ ]:
# Define bodies that are propagated
bodies_to_propagate = ["ISS"]

# Define central bodies of propagation
central_bodies = ["Earth"]

In [ ]:
# Define accelerations acting on ISS by Sun and Earth.
accelerations_settings_ISS = dict(
    Sun=[
        propagation_setup.acceleration.radiation_pressure(),
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Earth=[
        propagation_setup.acceleration.spherical_harmonic_gravity(50, 50),
        propagation_setup.acceleration.aerodynamic()
    ],
    Moon=[
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Mars=[
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Venus=[
        propagation_setup.acceleration.point_mass_gravity()
    ]
)



In [ ]:
# Create global accelerations settings dictionary.
acceleration_settings = {"ISS": accelerations_settings_ISS}

# Create acceleration models.
acceleration_models = propagation_setup.create_acceleration_models(
    bodies,
    acceleration_settings,
    bodies_to_propagate,
    central_bodies)

In [ ]:
from tudatpy.astro.time_conversion import DateTime, seconds_since_epoch_to_julian_day, julian_day_to_calendar_date

# Ask user for start and end date-time
start_datetime_str = input("Enter simulation start (YYYY-MM-DD HH:MM:SS): ")
end_datetime_str   = input("Enter simulation end   (YYYY-MM-DD HH:MM:SS): ")

# Parse into year, month, day, hour, minute, second
start_parts = list(map(int, start_datetime_str.replace(":", "-").replace(" ", "-").split("-")))
end_parts   = list(map(int, end_datetime_str.replace(":", "-").replace(" ", "-").split("-")))

# Create DateTime objects and convert to epoch
simulation_start_epoch = DateTime(*start_parts).epoch()
simulation_end_epoch   = DateTime(*end_parts).epoch()

# Convert epochs back to datetime objects for confirmation
def format_epoch(epoch_seconds):
    jd = seconds_since_epoch_to_julian_day(epoch_seconds)
    dt = julian_day_to_calendar_date(jd)  # This returns a datetime.datetime
    return dt.strftime("%Y-%m-%d %H:%M:%S UTC")

print(f"Simulation will run from {format_epoch(simulation_start_epoch)} (epoch={simulation_start_epoch:.1f})")
print(f"                        to {format_epoch(simulation_end_epoch)} (epoch={simulation_end_epoch:.1f})")


In [ ]:

ISS_tle = environment.Tle(
"1 25544U 98067A   25267.92252530  .00022724  00000-0  39583-3 0  9998",
    "2 25544  51.6334 172.7209 0004160  17.4034 342.7097 15.50645587530637",
)
# Create ephemeris based on the TLE
ISS_ephemeris = environment.TleEphemeris("Earth", "J2000", ISS_tle, False)

initial_state = ISS_ephemeris.cartesian_state( simulation_start_epoch )
print("Initial state vector [x, y, z, vx, vy, vz] in meters and m/s:")
print(initial_state)

In [ ]:
# Define list of dependent variables to save
dependent_variables_to_save = [
    propagation_setup.dependent_variable.total_acceleration("ISS"),
    propagation_setup.dependent_variable.keplerian_state("ISS", "Earth"),
    propagation_setup.dependent_variable.latitude("ISS", "Earth"),
    propagation_setup.dependent_variable.longitude("ISS", "Earth"),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.point_mass_gravity_type, "ISS", "Sun"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.point_mass_gravity_type, "ISS", "Moon"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.point_mass_gravity_type, "ISS", "Mars"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.point_mass_gravity_type, "ISS", "Venus"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.spherical_harmonic_gravity_type, "ISS", "Earth"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.aerodynamic_type, "ISS", "Earth"
    ),
    propagation_setup.dependent_variable.single_acceleration_norm(
        propagation_setup.acceleration.radiation_pressure_type, "ISS", "Sun"
    )
]

In [ ]:
# Adaptive RKF78 integrator settings
integrator_settings = propagation_setup.integrator.runge_kutta_variable_step_size(
    initial_time_step = 10,   # starting guess for step size [s]
    minimum_step_size = 0.001,    # lower bound [s]
    maximum_step_size = 10,  # upper bound [s]
    relative_error_tolerance = 1.0E-12,  # relative error tolerance
    absolute_error_tolerance = 1.0E-12,  # absolute error tolerance
    coefficient_set = propagation_setup.integrator.CoefficientSets.rkf_78
)
"""
integrator_settings = propagation_setup.integrator.adams_bashforth_moulton(
    initial_time_step = 1.0,
    minimum_step_size = 0.001,
    maximum_step_size = 10.0,
    relative_error_tolerance = 1.0E-12,
    absolute_error_tolerance = 1.0E-12,
    minimum_order = 6,
    maximum_order = 11
)"""



In [ ]:
# Create termination settings
termination_condition = propagation_setup.propagator.time_termination(simulation_end_epoch)
# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_state,
    simulation_start_epoch,
    integrator_settings,
    termination_condition,
    output_variables=dependent_variables_to_save
)

In [ ]:
# Create simulation object and propagate the dynamics
dynamics_simulator = numerical_simulation.create_dynamics_simulator(
    bodies, propagator_settings
)

# Extract the resulting state and dependent variable history and convert it to an ndarray
states = dynamics_simulator.propagation_results.state_history
states_array = result2array(states)
dep_vars = dynamics_simulator.propagation_results.dependent_variable_history
dep_vars_array = result2array(dep_vars)

In [ ]:
# Plot total acceleration as function of time
time_hours = (dep_vars_array[:,0] - dep_vars_array[0,0])/3600
total_acceleration_norm = np.linalg.norm(dep_vars_array[:,1:4], axis=1)
plt.figure(figsize=(9, 5))
plt.title("Total acceleration norm on ISS over the course of propagation.")
plt.plot(time_hours, total_acceleration_norm)
plt.xlabel('Time [hr]')
plt.ylabel('Total Acceleration [m/s$^2$]')
plt.xlim([min(time_hours), max(time_hours)])
plt.grid()
plt.tight_layout()

In [ ]:
# Plot ground track for a period of 3 hours
latitude = dep_vars_array[:,10]
longitude = dep_vars_array[:,11]

# Select 3 hours from the start
hours = 3
subset = int(len(time_hours) / 24 * hours)   # proportion for 3 hours
latitude = np.rad2deg(latitude[0: subset])
longitude = np.rad2deg(longitude[0: subset])

# Wrap longitude to [-180, 180] degrees
longitude = (longitude + 180) % 360 - 180

# Plot ground track
plt.figure(figsize=(10, 5))
plt.title("3-hour ground track of ISS")
plt.scatter(longitude, latitude, s=1, c="blue")
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.xlim([-180, 180])
plt.ylim([-90, 90])
plt.yticks(np.arange(-90, 91, step=30))
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Convert state history to array
state_array = result2array(dynamics_simulator.state_history)

# Extract final state (last row)
final_time = state_array[-1, 0]     # Epoch in seconds since J2000
final_state = state_array[-1, 1:]   # [x, y, z, vx, vy, vz]

# Convert units: m → km, m/s → km/s
final_pos_km = final_state[0:3] / 1e3
final_vel_kms = final_state[3:6] / 1e3

print("Final Epoch [s since J2000]:", final_time)
print("Final State Vector:")
print("x  = {:.3f} km".format(final_pos_km[0]))
print("y  = {:.3f} km".format(final_pos_km[1]))
print("z  = {:.3f} km".format(final_pos_km[2]))
print("vx = {:.6f} km/s".format(final_vel_kms[0]))
print("vy = {:.6f} km/s".format(final_vel_kms[1]))
print("vz = {:.6f} km/s".format(final_vel_kms[2]))



In [ ]:
# Plot Kepler elements as a function of time
kepler_elements = dep_vars_array[:,4:10]
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(9, 12))
fig.suptitle('Evolution of Kepler elements over the course of the propagation.')

# Semi-major Axis
semi_major_axis = kepler_elements[:,0] / 1e3
ax1.plot(time_hours, semi_major_axis)
ax1.set_ylabel('Semi-major axis [km]')

# Eccentricity
eccentricity = kepler_elements[:,1]
ax2.plot(time_hours, eccentricity)
ax2.set_ylabel('Eccentricity [-]')

# Inclination
inclination = np.rad2deg(kepler_elements[:,2])
ax3.plot(time_hours, inclination)
ax3.set_ylabel('Inclination [deg]')

# Argument of Periapsis
argument_of_periapsis = np.rad2deg(kepler_elements[:,3])
ax4.plot(time_hours, argument_of_periapsis)
ax4.set_ylabel('Argument of Periapsis [deg]')

# Right Ascension of the Ascending Node
raan = np.rad2deg(kepler_elements[:,4])
ax5.plot(time_hours, raan)
ax5.set_ylabel('RAAN [deg]')

# True Anomaly
true_anomaly = np.rad2deg(kepler_elements[:,5])
ax6.scatter(time_hours, true_anomaly, s=1)
ax6.set_ylabel('True Anomaly [deg]')
ax6.set_yticks(np.arange(0, 361, step=60))

for ax in fig.get_axes():
    ax.set_xlabel('Time [hr]')
    ax.set_xlim([min(time_hours), max(time_hours)])
    ax.grid()
plt.tight_layout()